In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [5]:
path = ('/home/micatske/ml/covid-analysis/covid_daten_knime.csv')
 
df_country = pd.read_csv(path)
df_country.head()

,Continent,Country,Lat,Long,Date,cases,deaths,cases_lastday,deaths_lastday,increase_cases,increase_deaths,increase_cases_lastday,increase_cases_proz,Max_Date,actual_day,df_days
0,Asia,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0,0,0,0,0,0.0,2020-11-18,0,301
1,Asia,Afghanistan,33.93911,67.709953,2020-01-23,0,0,0,0,0,0,0,0.0,2020-11-18,0,300
2,Asia,Afghanistan,33.93911,67.709953,2020-01-24,0,0,0,0,0,0,0,0.0,2020-11-18,0,299
3,Asia,Afghanistan,33.93911,67.709953,2020-01-25,0,0,0,0,0,0,0,0.0,2020-11-18,0,298
4,Asia,Afghanistan,33.93911,67.709953,2020-01-26,0,0,0,0,0,0,0,0.0,2020-11-18,0,297


In [6]:
df_country_exp = pd.DataFrame()

df_country_list = df_country.groupby(['Country']).last().reset_index()
country_list = df_country_list['Country'].to_list()

In [7]:
df_country_period = df_country[df_country['df_days'] <= 90]
zeilen = df_country_period['cases'].count()

In [8]:
df_country_period

,Continent,Country,Lat,Long,Date,cases,deaths,cases_lastday,deaths_lastday,increase_cases,increase_deaths,increase_cases_lastday,increase_cases_proz,Max_Date,actual_day,df_days
211,Asia,Afghanistan,33.939110,67.709953,2020-08-20,37856,1385,37599,1375,257,10,0,0.000000,2020-11-18,0,90
212,Asia,Afghanistan,33.939110,67.709953,2020-08-21,37894,1385,37856,1385,38,0,257,-0.852140,2020-11-18,0,89
213,Asia,Afghanistan,33.939110,67.709953,2020-08-22,37953,1385,37894,1385,59,0,38,0.552632,2020-11-18,0,88
214,Asia,Afghanistan,33.939110,67.709953,2020-08-23,37999,1387,37953,1385,46,2,59,-0.220339,2020-11-18,0,87
215,Asia,Afghanistan,33.939110,67.709953,2020-08-24,38054,1389,37999,1387,55,2,46,0.195652,2020-11-18,0,86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64925,Africa,Zimbabwe,-19.015438,29.154857,2020-11-14,8786,257,8765,257,21,0,69,-0.695652,2020-11-18,0,4
64926,Africa,Zimbabwe,-19.015438,29.154857,2020-11-15,8829,257,8786,257,43,0,21,1.047619,2020-11-18,0,3
64927,Africa,Zimbabwe,-19.015438,29.154857,2020-11-16,8897,257,8829,257,68,0,43,0.581395,2020-11-18,0,2
64928,Africa,Zimbabwe,-19.015438,29.154857,2020-11-17,8945,260,8897,257,48,3,68,-0.294118,2020-11-18,0,1


In [9]:
#country_list =['Austria','Switzerland','Germany','Spain','Burkina Faso']
#country_list =['Greenland']

df_country_exp = pd.DataFrame()

for country in country_list:

        df_country_modell = df_country_period[df_country['Country'] == country]
      
        expdays = 30
            
        zeilen= df_country_modell['cases'].count()
        x = np.arange(1,zeilen+1)
        x_exp = np.arange(1, x.max() + expdays  )
        y = df_country_modell['cases'] 
        
        y_min = y.min()
        y = y - y_min


        def logistic_function(x,b,r,k):
             return np.exp((x-b)*r)/ (1+((np.exp((x-b)*r)-1)/k))
        
        try:
            popt, pcov = curve_fit(logistic_function, x, y  )
            y_exp = (logistic_function(x_exp, *popt))
            y_exp = y_exp + y_min
            
            datum = pd.date_range(start=df_country_modell['Date'].max(), periods=expdays)
            datum = pd.date_range(start=df_country_modell['Date'].min(),end=datum.max())
            datum = datum.strftime("%Y-%m-%d")
            df = pd.DataFrame({'day':x_exp, 'expected':y_exp,'datum':datum, 'Country' : country})

            print(country)
            df_country_exp = df_country_exp.append(df)

        except RuntimeError:
            print(''.join(country) +  ": Error - curve_fit Log failed")

df_country_exp.reset_index(drop=True, inplace=True)
print('finished')



/tmp/ipykernel_32002/1553652556.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_country_modell = df_country_period[df_country['Country'] == country]
/tmp/ipykernel_32002/1553652556.py:22: RuntimeWarning: overflow encountered in exp
  return np.exp((x-b)*r)/ (1+((np.exp((x-b)*r)-1)/k))
/tmp/ipykernel_32002/1553652556.py:22: RuntimeWarning: invalid value encountered in true_divide
  return np.exp((x-b)*r)/ (1+((np.exp((x-b)*r)-1)/k))
/home/micatske/ml/my_env/lib/python3.8/site-packages/scipy/optimize/minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


Afghanistan
Albania
Algeria
Andorra
Angola
Anguilla
Antigua and Barbuda
Argentina
Armenia
Aruba
Australia
Austria
Azerbaijan
Bahamas
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Belize
Benin
Bermuda
Bhutan
Bolivia
Bonaire, Sint Eustatius and Saba
Bosnia and Herzegovina
Botswana
Brazil
British Virgin Islands
Brunei
Bulgaria
Burkina Faso
Burma
Burundi
Cabo Verde
Cambodia
Cameroon
Canada
Cayman Islands
Central African Republic
Chad
Channel Islands
Chile
China
Colombia
Comoros
Congo (Brazzaville)
Congo (Kinshasa)
Costa Rica
Cote d'Ivoire
Croatia
Cuba
Curacao
Cyprus
Czechia
Denmark
Djibouti
Dominica
Dominican Republic
Ecuador
Egypt
El Salvador
Equatorial Guinea
Eritrea
Estonia
Eswatini
Ethiopia
Falkland Islands (Malvinas)
Faroe Islands
Fiji
Finland
France
French Guiana
French Polynesia
Gabon
Gambia
Georgia
Germany
Ghana
Gibraltar
Greece
Greenland
Grenada
Guadeloupe
Guatemala
Guinea
Guinea-Bissau
Guyana
Haiti
Holy See
Honduras
Hungary
Iceland
India
Indonesia
Iran
Iraq
Ireland
Isle of Man
Isra

-85.44502539376629 0.0733194881278292 2738.1665148879065


In [10]:
plt.plot(x,y)
plt.plot(x_exp,y_exp)
plt.show()
print( *popt)

In [11]:
df_country_exp.head()

,day,expected,datum,Country
0,1,38192.407821,2020-08-20,Afghanistan
1,2,38204.290239,2020-08-21,Afghanistan
2,3,38216.584385,2020-08-22,Afghanistan
3,4,38229.303954,2020-08-23,Afghanistan
4,5,38242.463052,2020-08-24,Afghanistan
